<a href="https://colab.research.google.com/github/CShorten/Small-Weaviate-Examples/blob/main/Title_Matching_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jul 28 12:47:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install weaviate-client==3.6 > /dev/null
!pip install datasets > /dev/null
!pip install sentence_transformers > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from getpass import getpass
import weaviate
from weaviate.wcs import WCS

my_creds = weaviate.auth.AuthClientPassword(username=input("User name: "), password=getpass("Password: "))

User name: connorshorten300@gmail.com
Password: ··········


In [ ]:
my_wcs = WCS(my_creds)

client = weaviate.Client("https://abstract-vectors.semi.network")

In [ ]:
from datasets import load_dataset

ArXiv_ML_papers = load_dataset("CShorten/ML-ArXiv-Papers")["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
titles = ArXiv_ML_papers["title"]
abstracts = ArXiv_ML_papers["abstract"]

In [ ]:
import sentence_transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder

model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
import time

# batch this to speed it up!!!!
start = time.time()
title_embeds = []
for i, title in enumerate(titles):
  if i % 20_000 == 19_999:
    print(i)
  title_embeds.append(model.encode(title))

print("Ran in: " + str(int(time.time() - start)) + " seconds.")

19999
39999
59999
79999
99999
Ran in: 841 seconds.


In [ ]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [ ]:
import numpy as np

idx_in_retrieval = {
    0: 0,
    1: 0
}
original_rank_counter = {}
for i in range(1,101):
  original_rank_counter[i] = 0

rerank_rank_counter = {}
for i in range(1,101):
  rerank_rank_counter[i] = 0

for i in range(len(titles)):
  nearVector = {
      "vector": title_embeds[i]
  }

  result = client.query.get("Abstract_Vector_MiniLM", "index").with_near_vector(nearVector).do() # can we get more than 100 here?
  
  # get a list of values from a list of dictionaries
  parsed_results = result["data"]["Get"]["Abstract_Vector_MiniLM"]
  original_index_list = [val["index"] for val in parsed_results] # CURRENTLY RETURNING 100 -- can we get more?
  if i in set(original_index_list):    
    idx_in_retrieval[1] += 1
    data_pairs = [[titles[i], abstracts[retrieval_idx]] for retrieval_idx in original_index_list]
    found_abstract = np.where(np.array(original_index_list) == i)[0][0]
    original_rank_counter[found_abstract+1] += 1
    cross_scores = cross_encoder.predict(data_pairs)
    cross_encoder_ranks = {}
    for j, score in enumerate(cross_scores):
      cross_encoder_ranks[original_index_list[j]] = score
    
    re_ranked_list = sorted(cross_encoder_ranks.items(), key = lambda item: item[1], reverse=True)
    re_rank_scores = [tup[0] for tup in re_ranked_list]
    found_with_re_rank = np.where(np.array(re_rank_scores) == i)[0][0]
    rerank_rank_counter[found_with_re_rank+1] += 1

    if i < 10:
      print(i)
      print("="*10)
      print(original_index_list)
      print("="*10)
      print(re_ranked_list)
      print("\n")
    
    if i % 10_000 == 9_999:
      print(i)
      

  else:
    idx_in_retrieval[0] += 1

In [ ]:
print(i)

83612


In [ ]:
print("How often the abstract was in the top 100 retrievals")
print(idx_in_retrieval)
print("\n")
print("Count of where the retrieval placed the matching abstract, (closest to 1 is best)")
print(original_rank_counter)
print("\n")
print("Count of where the cross-encoder placed the matching abstract, (closest to 1 is best)")
print(rerank_rank_counter)

How often the abstract was in the top 100 retrievals
{0: 22993, 1: 60619}


Count of where the retrieval placed the matching abstract, (closest to 1 is best)
{1: 23965, 2: 5889, 3: 3299, 4: 2406, 5: 1815, 6: 1438, 7: 1222, 8: 998, 9: 931, 10: 851, 11: 783, 12: 683, 13: 650, 14: 587, 15: 551, 16: 520, 17: 501, 18: 465, 19: 443, 20: 370, 21: 376, 22: 345, 23: 332, 24: 337, 25: 324, 26: 300, 27: 294, 28: 298, 29: 252, 30: 263, 31: 248, 32: 255, 33: 262, 34: 222, 35: 242, 36: 205, 37: 222, 38: 201, 39: 203, 40: 203, 41: 224, 42: 191, 43: 150, 44: 184, 45: 202, 46: 163, 47: 177, 48: 167, 49: 157, 50: 160, 51: 153, 52: 157, 53: 149, 54: 136, 55: 142, 56: 117, 57: 128, 58: 138, 59: 138, 60: 127, 61: 109, 62: 126, 63: 111, 64: 131, 65: 111, 66: 109, 67: 113, 68: 117, 69: 113, 70: 100, 71: 99, 72: 95, 73: 108, 74: 107, 75: 103, 76: 101, 77: 78, 78: 88, 79: 101, 80: 94, 81: 95, 82: 93, 83: 85, 84: 77, 85: 84, 86: 78, 87: 78, 88: 89, 89: 85, 90: 85, 91: 76, 92: 79, 93: 85, 94: 78, 95: 76, 96: 70,

# Small Sample Size of 50 Title-Abstract Pairs

In [ ]:
print("How often the abstract was in the top 100 retrievals")
print(idx_in_retrieval)
print("\n")
print("Count of where the retrieval placed the matching abstract, (closest to 1 is best)")
print(original_rank_counter)
print("\n")
print("Count of where the cross-encoder placed the matching abstract, (closest to 1 is best)")
print(rerank_rank_counter)

How often the abstract was in the top 100 retrievals
{0: 17, 1: 33}


Count of where the retrieval placed the matching abstract, (closest to 1 is best)
{1: 13, 2: 3, 3: 3, 4: 0, 5: 1, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 1, 37: 0, 38: 0, 39: 1, 40: 0, 41: 1, 42: 0, 43: 0, 44: 0, 45: 1, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 1, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 1, 69: 1, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 1, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0}


Count of where the cross-encoder placed the matching abstract, (closest to 1 is best)
{1: 26, 2: 5, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: